In [1]:
!pip install requests

In [21]:
import tensorflow as tf

# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import os
import requests

# Get Image Data (From `image_url` in `search_15k6.csv`)

In [16]:
df = pd.read_csv("data/search_15k6.csv",encoding='utf-8',sep='\t')
df['product_id'] = df.index
#df = df.set_index('product_id')

In [10]:
if not os.path.exists("data/images"):
    os.makedirs("data/images")

In [ ]:
for i in range(len(df)):
    item = df.iloc[i]
    category = item['category']
    image_url = item['image_url']
    product_id = item['product_id']
    print(str(product_id))
    if not os.path.exists(os.path.join("data/images", category)):
        os.makedirs(os.path.join("data/images", category))
    r = requests.get(image_url, allow_redirects=True) #get image from image_url
    filename = str(product_id) + '.png'
    open(os.path.join("data/images", category, filename), 'wb').write(r.content)
    

# Preprocess and split data into train - val - test set

In [ ]:
!rm -rf data/images/*/.ipynb_checkpoints
!rm -rf data/*/.ipynb_checkpoints

In [ ]:
files_dict = {}
for category in os.listdir("data/images/"):
    files_dict[category] = []
    files_dict[category] = [os.path.join(category,filename) for filename in os.listdir(os.path.join("data/images", category))]
print(files_dict.keys())
print(files_dict['DIENTHOAI'][0])

dict_keys(['VOBAOOPLUNG', 'CAPSAC', 'PHUKIEN', 'LINHKIENMAYTINH', 'PINSAC', 'OCUNG', 'CHUOTBANPHIM', 'GAYCHUPHINH', 'MAYTINHBANG', 'MAYBAN', 'AUDIO', 'DIENTHOAI', 'LAPTOP', 'SIM', 'THIETBIMANG', 'MIENGDAN'])
DIENTHOAI/908.png


In [ ]:
files_dict_train, files_dict_val, files_dict_test = {}, {}, {}
for category, file_list in files_dict.items():
    file_list_train, file_list_test = train_test_split(file_list, test_size=0.2, random_state = 1)
    file_list_train, file_list_val = train_test_split(file_list_train, test_size=0.1, random_state=1)
    
    files_dict_train[category] = file_list_train
    files_dict_val[category] = file_list_val
    files_dict_test[category] = file_list_test


In [ ]:
#write the splitted dataset's file names
with open("data/images_train.txt", "w") as f:
    for category,file_list in files_dict_train.items():
        for filename in file_list:
            f.write(filename+"\n")
            
with open("data/images_val.txt", "w") as f:
    for category,file_list in files_dict_val.items():
        for filename in file_list:
            f.write(filename+"\n")
            
with open("data/images_test.txt", "w") as f:
    for category,file_list in files_dict_test.items():
        for filename in file_list:
            f.write(filename+"\n")

In [ ]:
with open("data/images_train.txt") as f:
    train_file_list = f.read().splitlines()
    
with open("data/images_val.txt") as f:
    val_file_list = f.read().splitlines()
    
with open("data/images_test.txt") as f:
    test_file_list = f.read().splitlines()

In [ ]:
#create splitted dataset
if not os.path.exists("data/images_splitted"):
    os.makedirs("data/images_splitted")

In [ ]:
!rm -rf data/images_splitted/

In [ ]:
#create splitted train dataset
for file in train_file_list:
    f = open(os.path.join('data/images',file), 'rb')
    if f:
        category, filename = file.split("/")

        if not os.path.exists(os.path.join("data/images_splitted/train", category)):
            os.makedirs(os.path.join("data/images_splitted/train", category))
        open(os.path.join("data/images_splitted/train", file), 'wb').write(f.read()) #write to new destination
    
#create splitted val dataset
for file in val_file_list:
    f = open(os.path.join('data/images',file), 'rb')
    if f:
        category, filename = file.split("/")

        if not os.path.exists(os.path.join("data/images_splitted/val", category)):
            os.makedirs(os.path.join("data/images_splitted/val", category))
        open(os.path.join("data/images_splitted/val", file), 'wb').write(f.read()) #write to new destination
        
#create splitted test dataset        
for file in test_file_list:
    f = open(os.path.join('data/images',file), 'rb')
    if f:
        category, filename = file.split("/")

        if not os.path.exists(os.path.join("data/images_splitted/test", category)):
            os.makedirs(os.path.join("data/images_splitted/test", category))
        open(os.path.join("data/images_splitted/test", file), 'wb').write(f.read()) #write to new destination
        

In [ ]:
#Remove corrupted images
def remove_corrupted_images(subset):
    for category in os.listdir(f"data/images_splitted/{subset}/"):
        for i in os.listdir(f"data/images_splitted/{subset}/"+category):
            img_path = f"data/images_splitted/{subset}/"+category+'/'+i

            img = cv2.imread(img_path)

            if img is None:
                print(img_path,end=' ')
                os.remove(img_path)
                print("Remove!")
    #         else:
    #             print(img.shape)


In [ ]:
remove_corrupted_images("train")
remove_corrupted_images("val")
remove_corrupted_images("test")

data/images_splitted/train/PINSAC/3936.png Remove!
data/images_splitted/train/PINSAC/3935.png Remove!
data/images_splitted/train/AUDIO/14801.png Remove!


In [ ]:
!rm -rf data/images_splitted/*/*/.ipynb_checkpoints

# Image Product Category Prediction

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255)



In [ ]:
#Using ResNet50 pretrained on ImageNet
backbone = tf.keras.applications.ResNet50(include_top=False,weights='imagenet', input_shape=(224,224,3), pooling='avg')
x = tf.keras.layers.Dense(16, activation='softmax')(backbone.output)
model = tf.keras.Model(inputs=backbone.input, outputs =x)

In [ ]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
train = train_datagen.flow_from_directory("data/images_splitted/train",target_size=(224,224),class_mode='categorical',batch_size=32)
val = val_datagen.flow_from_directory("data/images_splitted/val",target_size=(224,224),class_mode='categorical',batch_size=32)

Found 11261 images belonging to 16 classes.
Found 1253 images belonging to 16 classes.


In [ ]:
model.compile(optimizer = tf.optimizers.Adam(lr=1e-4),
              loss = 'categorical_crossentropy',
              metrics=['acc'])

In [ ]:
if not os.path.exists("models"):
    os.makedirs("models")
    
mcp_save = tf.keras.callbacks.ModelCheckpoint(
    filepath="models/ResNet50_product_16.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True) 

history = model.fit(train, 
                    epochs=50,
                    validation_data=val,
                    callbacks=[mcp_save])


Epoch 1/50
352/352 [==============================] - 97s 276ms/step - loss: 1.2520 - acc: 0.6231 - val_loss: 3.2081 - val_acc: 0.0950
Epoch 2/50
352/352 [==============================] - 96s 272ms/step - loss: 0.7568 - acc: 0.7671 - val_loss: 1.5380 - val_acc: 0.5467
Epoch 3/50
352/352 [==============================] - 96s 273ms/step - loss: 0.5513 - acc: 0.8246 - val_loss: 1.2221 - val_acc: 0.6752
Epoch 4/50
352/352 [==============================] - 96s 273ms/step - loss: 0.3991 - acc: 0.8745 - val_loss: 1.1036 - val_acc: 0.7167
Epoch 5/50
352/352 [==============================] - 95s 271ms/step - loss: 0.3166 - acc: 0.8993 - val_loss: 1.2689 - val_acc: 0.6808
Epoch 6/50
352/352 [==============================] - 95s 271ms/step - loss: 0.2655 - acc: 0.9187 - val_loss: 1.2650 - val_acc: 0.7015
Epoch 7/50
352/352 [==============================] - 95s 271ms/step - loss: 0.2188 - acc: 0.9334 - val_loss: 1.2764 - val_acc: 0.6983
Epoch 8/50
352/352 [==============================] - 9

In [ ]:
model.load_weights("models/ResNet50_product_16.h5")


In [ ]:
model.evaluate(val_datagen.flow_from_directory("data/images_splitted/train",target_size=(224,224),class_mode='categorical',batch_size=32))

Found 11261 images belonging to 16 classes.
352/352 [==============================] - 22s 62ms/step - loss: 0.0449 - acc: 0.9853


[0.044933758676052094, 0.9853476881980896]

In [ ]:
model.evaluate(val_datagen.flow_from_directory("data/images_splitted/val",target_size=(224,224),class_mode='categorical',batch_size=32))

Found 1253 images belonging to 16 classes.
40/40 [==============================] - 2s 59ms/step - loss: 1.4577 - acc: 0.7590


[1.4577085971832275, 0.7589784264564514]

In [ ]:
model.evaluate(val_datagen.flow_from_directory("data/images_splitted/test",target_size=(224,224),class_mode='categorical',batch_size=32))

Found 3131 images belonging to 16 classes.
98/98 [==============================] - 6s 61ms/step - loss: 1.4004 - acc: 0.7614


[1.4003503322601318, 0.7614181041717529]

In [ ]:
class_names = sorted(os.listdir('data/images_splitted/test'))

In [ ]:
img = cv2.imread("test.jpg")[:,:,::-1]

In [ ]:
plt.figure()
plt.imshow(img)

In [ ]:

scores = model.predict(val_datagen.flow(cv2.resize(img,(224,224))[None,:,:]))
print(scores)
class_names[int(tf.argmax(scores,axis=1).numpy())]

[[3.2670401e-02 2.2601662e-06 3.4040983e-03 2.9648101e-02 7.4045857e-07
  4.4164830e-01 4.5887445e-04 3.0155340e-02 7.3454394e-06 4.5212608e-02
  3.3964712e-05 1.4091099e-02 3.1366605e-01 2.0033276e-05 1.8599387e-02
  7.0381455e-02]]


'LAPTOP'

In [ ]:
train.class_indices

{'AUDIO': 0,
 'CAPSAC': 1,
 'CHUOTBANPHIM': 2,
 'DIENTHOAI': 3,
 'GAYCHUPHINH': 4,
 'LAPTOP': 5,
 'LINHKIENMAYTINH': 6,
 'MAYBAN': 7,
 'MAYTINHBANG': 8,
 'MIENGDAN': 9,
 'OCUNG': 10,
 'PHUKIEN': 11,
 'PINSAC': 12,
 'SIM': 13,
 'THIETBIMANG': 14,
 'VOBAOOPLUNG': 15}